# Produced Gas Get Pi Data and  Produced Gas Generate Visuals and Create Tables
Original Author: *Eric Jack*  <br> Modified by: *Monique Beaulieu*

- Query PI data (via AF SDK) for tags associated with either pads or individual wells

- Save this data to .pkl files so it can be reused and not constantly pulled

- Provide base data for plotting:

    - SRU & emulsion plots

    -  Well status frequency

    -  Other pad-level trends

Builds on *py_get_tag_data.ipynb* and generates visuals and analysis tables for a daily Produced Gas report:
- Primarily using pad-level produced gas (PG) and well frequency data.
- Uses pad-level PG (df_all_data_pads_pg) for ROC and plotting
- Pulls well-level data: esp_frequency, temp_tubing, and casing_valve
<br>


Gives a high-level overview of how field emulsion compares with total gas production and constraints 

TODO:
- see if code runs locally
- create a dataframe with all the data
- create logic to decide the pg on a well by well level
- make a visual per well? show percentile values- avg, max, min etc?

## Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import dateutil.relativedelta
import plotly.graph_objects as go
import traceback
import clr
import sys
import json
import string
import time
import plotly.io as pio

from cmath import nan
from unicodedata import name
from scipy.stats import linregress
from plotly.subplots import make_subplots
from pyparsing import line

- Global rootDir and time parameters for the data window: endDate is today at 5:55 AM.
- Initialize a basic logging system (Log.txt) to track script starts/completions/errors.

Log for when this script is run

In [2]:
rootDir = os.getcwd()
print("Running from:", rootDir)
pio.kaleido.scope.mathjax = None


with open(os.path.join(rootDir, "Log.txt"), "a+") as f:
    f.write("\n\n-----------------------------------\n")
    f.write(f"{dt.datetime.today()} - Starting Code\n")

Running from: c:\Users\MoBeaulieu\OneDrive - Suncor Energy Inc\Documents\python_projects_local\pg_script



- OSIsorft PI AF SDK used to pull time-series data from PI System
- Root directory where pickle files and tag excel sheets are stored

In [3]:
sys.path.append(r"C:\Program Files (x86)\PIPC\AF\PublicAssemblies\4.0")
clr.AddReference('OSIsoft.AFSDK')

from OSIsoft.AF import *
from OSIsoft.AF.PI import *
from OSIsoft.AF.Asset import *
from OSIsoft.AF.Data import *
from OSIsoft.AF.Time import *
from OSIsoft.AF.UnitsOfMeasure import *

piServers = PIServers()
piServer = piServers["firebagpi"]


## Global Pi Time Settings
This defines the global time window (last 14 days at 6-minute intervals) that all data pulls, unless redefined.



In [12]:
#########PI DATA SETUP#############

endDate = dt.datetime.combine(dt.date.today (), dt.time(hour=5, minute=55))
print(endDate)

parseTime = '.1h'
span = AFTimeSpan.Parse(parseTime)

#endDate = dt.date.today()
startDate = (endDate - dateutil.relativedelta.relativedelta(days = 14))
timeRange = AFTimeRange(str(startDate), str(endDate))

piServers = PIServers()    
piServer = piServers["firebagpi"]
######################################

2025-04-21 05:55:00


## Utility Functions

In [4]:

def get_PI_data(tag, col):
	""" 
	Fetches interpolated data from PI for a specific tag and does some basic data cleaning based on column type.
		- Gets the PI tag point.
		- Pulls interpolated values over timeRange at a frequency defined by span (set globally to .1h, or 6 min in  function_get_pi_data_create_pickle_P9192()).
		- Validates values:
    		- If it's temp_tubing, filters out garbage values (outside 0–300°C).
		- Returns two lists:
			- data (numerical values or None)
			- date (timestamps)
	"""
	print(tag)
	pt = PIPoint.FindPIPoint(piServer,tag.replace(" ",""))

	#pulls interpolated data between timerange and at given frequency defined by span
	interpolated = pt.InterpolatedValues(timeRange, span, "", False)

	#creates lists to store data and date associated with that PI tag
	data = []
	date = []     

	#appends data and date to lists 
	for event in interpolated:
		try:
			float(event.Value)
			tagVal = event.Value
			if col == 'esp_frequency':
				newVal = event.Value
				
			if col == 'temp_tubing':
				if tagVal >=0 and tagVal <= 300:
					newVal = event.Value
				else:
					newVal = None			
			else:
				newVal = tagVal
			data.append(newVal)
			date.append(event.Timestamp.LocalTime)
		except:
			data.append(None)
			date.append(event.Timestamp.LocalTime)
			continue

	#changes the date format/type
	try:
		date = [dt.datetime.strptime(str(date), '%m/%d/%Y %I:%M:%S %p') for date in date]
	except:
		date = [dt.datetime.strptime(str(date), '%Y-%m-%d %I:%M:%S %p') for date in date]
	return(data, date)


In [5]:
def function_get_pi_data_create_pickle_wells(pad):
	"""
	Pulls well-level data from PI and saves to a .pkl file.
		- Loads the tags_well sheet from Tags.xlsx (which must include pad, well, and PI tag names).
		- For each well-tag:
			- Uses get_PI_data() to fetch time series
			- Creates a tidy dataframe with: pad, well, date, value, and attribute (e.g., esp_frequency, temp_tubing)
		- Concatenates all well data and saves as: /prod_pickle_files/data_wells_<pad>.pkl
	"""

	#df_tags= pd.read_excel (rootDir + '/Tags.xlsx', sheet_name='tags_well')
	df_tags = pd.read_excel(os.path.join(rootDir, "Tags.xlsx"), sheet_name='tags_well')
	print(df_tags)
	df_tags = df_tags.loc[df_tags['pad']==pad]
	print(df_tags)
	cols = df_tags.iloc[:,2:].columns
	df_all_data = pd.DataFrame(columns=['pad', 'well', 'date', 'value', 'attribute'])

	for index, row in df_tags.iterrows():
		print(row['well'])
		for col in cols:
			tag = row[col]

			if tag != "NO TAG":

				tag_data_from_PI = get_PI_data(tag,col)
				date = tag_data_from_PI[1]
				values = tag_data_from_PI[0]

				count = len(date)

				temp_padList = [row['pad']]*count
				temp_wellList = [row['well']]*count
				temp_attributeList = [col]*count

				temp_df = pd.DataFrame(data={'pad':temp_padList, 'well':temp_wellList, 'date': date, 'value': values, 'attribute':temp_attributeList})
				df_all_data = pd.concat([df_all_data, temp_df])
				#df_all_data = df_all_data.append(temp_df)

	print(df_all_data)
	df_all_data = df_all_data.reset_index()
	df_all_data['value'] = pd.to_numeric(df_all_data['value'], errors='coerce')

	df_all_data.to_pickle(rootDir + "/prod_pickle_files/data_wells_" + pad + ".pkl")


In [6]:
def function_get_pi_data_create_pickle_pads(pad):
	"""
	Pulls pad-level data (produced_gas) from PI and saves to a .pkl file.
		- Loads the tags_pad sheet from Tags.xlsx.
		- For each pad-tag:
			- Uses get_PI_data() to fetch time series
			- Creates a tidy dataframe with: pad, date, value, and attribute
		- Concatenates all pad data and saves as: /prod_pickle_files/data_pads_<pad>.pkl
	"""
	# df_tags= pd.read_excel (rootDir + '/Tags.xlsx', sheet_name='tags_pad')
	df_tags = pd.read_excel(os.path.join(rootDir, "Tags.xlsx"), sheet_name='tags_pad')

	df_tags = df_tags.loc[df_tags['pad']==pad]
	cols = df_tags.iloc[:,1:].columns
	df_all_data = pd.DataFrame(columns=['pad', 'date', 'value', 'attribute'])

	for index, row in df_tags.iterrows():
		print(row['pad'])
		for col in cols:
			tag = row[col]

			if tag != "NO TAG":

				tag_data_from_PI = get_PI_data(tag,col)
				date = tag_data_from_PI[1]
				values = tag_data_from_PI[0]

				count = len(date)

				temp_padList = [row['pad']]*count
				temp_attributeList = [col]*count

				temp_df = pd.DataFrame(data={'pad':temp_padList, 'date': date, 'value': values, 'attribute':temp_attributeList})
				df_all_data = pd.concat([df_all_data, temp_df])
				#df_all_data = df_all_data.append(temp_df)

	print(df_all_data)
	df_all_data = df_all_data.reset_index()
	df_all_data['value'] = pd.to_numeric(df_all_data['value'], errors='coerce')

	df_all_data.to_pickle(rootDir + "/prod_pickle_files/data_pads_" + pad + ".pkl")



In [7]:
def function_get_pi_data_create_pickle_P9192 ():
	"""
	Specifically tailored for P91 and P92 pads, where data is stored separately.
		- Hardcodes the tags for P91 and P92 flow meters.
		- Pulls 14-day interpolated data for both.
		- Sums them into a single column gas_sum.
		- Saves as:
	"""	
	
	meaurement_points = {
		'p91': '91FI-14001/PV.CV',
		'p92': '92FI-1022/PV.CV',
	}

	parseTime = '.1h'
	span = AFTimeSpan.Parse(parseTime)

	#endDate = dt.date.today()
	startDate = (endDate - dateutil.relativedelta.relativedelta(days = 14))
	timeRange = AFTimeRange(str(startDate), str(endDate))

	piServers = PIServers()    
	piServer = piServers["firebagpi"]

	df_P9192_data = pd.DataFrame()
	tagCount = 0
	for key in meaurement_points:
		print(key)
		print(meaurement_points[key])
		tag = meaurement_points[key]

		print(tag)
		pt = PIPoint.FindPIPoint(piServer,tag.replace(" ",""))

		#pulls interpolated data between timerange and at given frequency defined by span
		interpolated = pt.InterpolatedValues(timeRange, span, "", False)

		#creates lists to store data and date associated with that PI tag
		data = []
		date = []     

		#appends data and date to lists 
		for event in interpolated:
			try:
				float(event.Value)
				tagVal = event.Value
				data.append(tagVal)
				date.append(event.Timestamp.LocalTime)
			except:
				data.append(None)
				date.append(event.Timestamp.LocalTime)
				continue

		#changes the date format/type
		try:
			date = [dt.datetime.strptime(str(date), '%m/%d/%Y %I:%M:%S %p') for date in date]
		except:
			date = [dt.datetime.strptime(str(date), '%Y-%m-%d %I:%M:%S %p') for date in date]
		
		if tagCount ==0:
			df_P9192_data['date'] = date
			df_P9192_data[key] = data
		else:
			df_P9192_data[key] = data

		tagCount = tagCount+1

	df_P9192_data['gas_sum'] = df_P9192_data[['p91', 'p92']].sum(axis=1)

	countOfVals = len(df_P9192_data['gas_sum'].tolist())
	temp_padList = ['P91_92'] * countOfVals
	temp_attributeList = ['produced_gas']* countOfVals

	df_to_pickle = pd.DataFrame(data={'pad':temp_padList, 'date': df_P9192_data['date'].tolist(), 'value': df_P9192_data['gas_sum'].tolist(), 'attribute':temp_attributeList})
	df_to_pickle.to_pickle(rootDir + "/prod_pickle_files/data_pads_P91_92.pkl")



In [8]:


def create_sru_plot_png ():
	"""
	Pulls multiple tags for SRU (Sulphur Recovery Unit) and field data and builds a DataFrame.
		- Tags include:
			- Multiple plant meters (plant_1 ... plant_7)
			- sru (SRU flow)
			- field_emul (field emulsion)
		- Pulls data using same PI logic as above.
			- Sums plant meters into plant_sum.
			- Calculates res_gas = sru - plant_sum
			- Returns a DataFrame with: date, plant_1...plant_7, sru, field_emul, plant_sum, res_gas
	"""
	parseTime = '.1h'
	span = AFTimeSpan.Parse(parseTime)

	#endDate = dt.date.today()
	#endDate = dt.datetime(2022, 4, 14, 6, 45, 0, 0)
	startDate = (endDate - dateutil.relativedelta.relativedelta(days = 14))
	timeRange = AFTimeRange(str(startDate), str(endDate))

	piServers = PIServers()    
	piServer = piServers["firebagpi"]

	df_sru_plot_data = pd.DataFrame()

	sru_plot_tags_dict = {
		'plant_1': '93FI-81150/PV.CV',
		'plant_2': '99FI-40559/ALM1/PV.CV',
		'plant_3': '93FI-22203/ALM1/PV.CV',
		'plant_4': '92FI-2020/PV.CV',
		'plant_5': '91FI-47408/PV.CV',
		'plant_6': '91FI-13001/PV.CV',
		'plant_7': '91FI-27408/PV.CV',
		'sru': '91FC-1019/PID1/PV.CV',
		'field_emul': 'FB_TOTAL_EMULSION_CORRECTED',
	}
	
	tagCount = 0
	for key in sru_plot_tags_dict:
		print(key)
		print(sru_plot_tags_dict[key])
		tag = sru_plot_tags_dict[key]



		print(tag)
		pt = PIPoint.FindPIPoint(piServer,tag.replace(" ",""))

		#pulls interpolated data between timerange and at given frequency defined by span
		interpolated = pt.InterpolatedValues(timeRange, span, "", False)

		#creates lists to store data and date associated with that PI tag
		data = []
		date = []     

		#appends data and date to lists 
		for event in interpolated:
			try:
				float(event.Value)
				tagVal = event.Value
				data.append(tagVal)
				date.append(event.Timestamp.LocalTime)
			except:
				data.append(None)
				date.append(event.Timestamp.LocalTime)
				continue

		#changes the date format/type
		try:
			date = [dt.datetime.strptime(str(date), '%m/%d/%Y %I:%M:%S %p') for date in date]
		except:
			date = [dt.datetime.strptime(str(date), '%Y-%m-%d %I:%M:%S %p') for date in date]
		
		if tagCount ==0:
			df_sru_plot_data['date'] = date
			df_sru_plot_data[key] = data
		else:
			df_sru_plot_data[key] = data

		tagCount = tagCount+1

	df_sru_plot_data['plant_sum'] = df_sru_plot_data[['plant_1', 'plant_2', 'plant_3', 'plant_4', 'plant_5', 'plant_6', 'plant_7']].sum(axis=1)
	df_sru_plot_data['res_gas'] = df_sru_plot_data['sru'] - df_sru_plot_data['plant_sum']

	return df_sru_plot_data

In [9]:
		
def create_cut_wells_status_data ():
	"""
	Used for visualization of high gas wells’ operating frequency (min, max, current).
		- Reads high_gas_offenders sheet from Tags.xlsx.
		- For each well:
			- Pulls latest ESP frequency
			- Stores: well, low_freq, high_freq, current_freq
		- Returns a DataFrame for plotting.
	"""

	# df_well_info= pd.read_excel (rootDir + '/Tags.xlsx', sheet_name='high_gas_offenders')
	df_well_info = pd.read_excel(os.path.join(rootDir, "Tags.xlsx"), sheet_name='high_gas_offenders')

	print(df_well_info)


	parseTime = '.1h'
	span = AFTimeSpan.Parse(parseTime)

	#endDate = dt.date.today()
	startDate = endDate 
	timeRange = AFTimeRange(str(startDate), str(endDate))

	piServers = PIServers()    
	piServer = piServers["firebagpi"]


	df_all_data = pd.DataFrame(columns=['well', 'low_freq', 'high_freq', 'current_freq'])

	for index, row in df_well_info.iterrows():
		well = row['well']
		esp_freq_tag = row['esp_freq_tag']
		print(esp_freq_tag)

		df_temp = pd.DataFrame()
		df_temp['well'] = well

		pt = PIPoint.FindPIPoint(piServer,esp_freq_tag.replace(" ",""))

		interpolated = pt.InterpolatedValues(timeRange, span, "", False)

		#creates lists to store data associated with that PI tag
		data = []    
		#appends data  to lists 
		for event in interpolated:
			try:
				float(event.Value)
				tagVal = event.Value
				data.append(tagVal)
			except:
				data.append(0)
		print(data)
		freq_val = round(data[0],1)
		temp_df = pd.DataFrame(data={'well':[well], 'low_freq':row['low'], 'high_freq': row['high'], 'current_freq':[freq_val]})
		df_all_data = pd.concat([df_all_data, temp_df])

	return df_all_data


## Create Images for Report

- Trips = red triangle
- Starts - green triangle
- NFEs = blue X 

In [14]:

try:
	#attempts to create df out of pickle. If not exists it will pull data and recreate pickle
	print('in try')
	#padInput = input("Enter pad number ex. '103' and hit enter.")

	rootDir = r"C:\Users\MoBeaulieu\OneDrive - Suncor Energy Inc\Documents\python_projects_local\pg_script"

	endDate = dt.datetime.combine(dt.date.today (), dt.time(hour=5, minute=55))
	pg_roc_lookback_list = [2,7,14]

	df_pg_roc_data = pd.DataFrame()
	df_pg_roc_data['mp'] = ''
	for i in pg_roc_lookback_list:
		df_pg_roc_data[str(i) + 'd'] = nan


	
	# get SRU, plant process gas, and field emulsion data for SRU summary plot
	df = create_sru_plot_png() # Pulls emulsion, plant gas, and SRU gas data.
	# Plots: Emulsion on a secondary y-axis, Plant gas and reservoir gas on teh primary y-axis, Constraint lines at 12,700 and 10,500 m3/h
	
	print(df)

	fig_sru_plot = make_subplots(specs=[[{"secondary_y": True}]])

	fig_sru_plot.add_trace(go.Scatter(

		x=df['date'].tolist(), y=df['field_emul'].tolist(),
		hoverinfo='x+y',
		mode='lines',
		name = 'Emulsion (2nd)',
		line=dict(width=1.5, color='rgb(0, 0, 0)', ),
		),
		secondary_y = True
	)

	fig_sru_plot.add_trace(go.Scatter(

		x=df['date'].tolist(), y=df['plant_sum'].tolist(),
		hoverinfo='x+y',
		mode='lines',
		name = 'Plant Gas (1st)',
		line=dict(width=1, color='blue'),
		stackgroup='pg'
		), # define stack group
		secondary_y = False
	)
	fig_sru_plot.add_trace(go.Scatter(

		x=df['date'].tolist(), y=df['res_gas'].tolist(),
		hoverinfo='x+y',
		mode='lines',
		name = 'Reservoir Gas (1st)',
		line=dict(width=1, color='red'),
		stackgroup='pg'
		), # define stack group
		secondary_y = False
	)



	sru_constraint_data = [12700]*len(df['date'].tolist())

	fig_sru_plot.add_trace(go.Scatter(
		x=df['date'].tolist(), y=sru_constraint_data,
		hoverinfo='x+y',
		mode='lines',
		name = 'SRU Constraint (1st)',
		line=dict(width=5, color='darkred', dash='dash'),
		), # define stack group
		secondary_y = False
	)

	sru_constraint_data_2 = [10500]*len(df['date'].tolist())

	fig_sru_plot.add_trace(go.Scatter(
		x=df['date'].tolist(), y=sru_constraint_data_2,
		hoverinfo='x+y',
		mode='lines',
		name = 'SRU Constraint (1st)',
		line=dict(width=5, color='orange', dash='dash'),
		), # define stack group
		secondary_y = False
	)

	fig_sru_plot.update_yaxes(title_text="Produced Gas (m3/h)",secondary_y=False)
	fig_sru_plot.update_yaxes(title_text="Emulsion (m3/h)",secondary_y=True)

	# fig_sru_plot.update_yaxes(title_text="Produced Gas (m3/h)", titlefont = dict(size = 20), tickfont = dict(size=20), secondary_y=False)
	# fig_sru_plot.update_yaxes(title_text="Emulsion (m3/h)",titlefont = dict(size = 20), tickfont = dict(size=20), secondary_y=True)
	#fig_sru_plot.update_layout(yaxis_range=(0, 100))

	fig_sru_plot.update_layout(width=1200, height = 500, margin=dict(l= 0, r= 0, t=0, b=0))
	fig_sru_plot.update_layout(legend=dict(yanchor="top", y=0.20, xanchor="left", x=0))
	#fig_sru_plot.write_image(rootDir + "/prod_report_images/sru_plot.png", format="png", scale=3, engine="kaleido") 

#############################################################################################################################################################################################
	
	print('about to enter get well status data')
	# well status vizualization
	df = create_cut_wells_status_data()
	# Loads a list of "high gas offender" wells from Excel
	# Pulls current ESP frequency values and compares to low/high thresholds
	# Plots markers: Red for low/high bounds, Black for current value (only if active)

	print(df)
	print('well status data df should be above')

	fig_well_cuts = go.Figure()

	for index, row in df.iterrows():

		well = row['well']
		low = row['low_freq']
		high = row['high_freq']
		curr = row['current_freq']

		
		if curr < 10:
			x_text = well + '<br>(offline)'
		
		else:
			x_text = well



		fig_well_cuts.add_trace(go.Scatter(
			x=[x_text], y=[high],
			mode='markers+text',
			marker=dict(
				color='red',
				size=15,
				line = dict(
					color = 'red',
					width = 4
				),
			),
			text = [high],
			textposition="top center",
			textfont=dict(
				family="sans serif",
				size=20,
				color='red'
			),
			marker_symbol = 'line-ew', 
			name=well + ' high', 
			legendgroup = well)
		)

		fig_well_cuts.add_trace(go.Scatter(
			x=[x_text], y=[low],
			mode='markers+text',
			marker=dict(
				color='red',
				size=15,
				line = dict(
					color = 'red',
					width = 4
				),
			),
			text = [low],
			textposition="bottom center",
			textfont=dict(
				family="sans serif",
				size=20,
				color='red'
			),
			marker_symbol = 'line-ew', 
			name=well + ' low', 
			legendgroup = well)
		)

		#adding this trace at the end so it is over top of the high/low markers. 
		if curr >= 10:
			fig_well_cuts.add_trace(go.Scatter(
				x=[x_text], y=[curr],
				mode='markers+text',
				marker=dict(
					color='black',
					size=20,
					line = dict(
						color = 'black',
						width = 4
					),
				),
				text = [curr],
				textposition="middle right",
				textfont=dict(
					family="sans serif",
					size=20,
					color='black'
				),
				marker_symbol = 'line-ew', 
				name=well + ' current', 
				legendgroup = well)
			)


	fig_well_cuts.update_yaxes(title_text="Freq (hz)") #, titlefont = dict(size = 20), tickfont = dict(size=20))
	fig_well_cuts.update_xaxes() #tickfont = dict(size=15))
	fig_well_cuts.update_layout(width=1500, height = 180, showlegend=False, margin=dict(l= 0, r= 0, t=0, b=0))
	fig_well_cuts.update_yaxes(range=[30, 65])
	#fig_well_cuts.write_image(rootDir + "/prod_report_images/well_cuts_plot.png", format="png", scale=3, engine="kaleido") 

	#############################################################################################################################################################################################

	measurementPoints = ['P91_92', 'P105', 'P106', 'P107', 'P108', 'P110', 'P114', 'P115', 'P116', 'P117', 'P112', 'P121']
	#measurementPoints = ['P106']

	all_events = []
	# Main processing loop over pads
	for mp in measurementPoints:
		# Loads pad and well pickle files (or regenerates them if missing)
		# Filters data into: esp_frequency (to detect trips, starts, speedups), temp_tubing (to detect deadhead/NFE events), and casing_valve (to get 48hr avg casing valve position)

		wellsPickleName = 'data_wells_'+ str(mp) + '.pkl'
		padsPickleName = 'data_pads_'+ str(mp) + '.pkl'


		prodMode = False
		if prodMode == True:
			function_get_pi_data_create_pickle_wells(str(mp))
			df_all_data_wells = pd.read_pickle(rootDir + "/prod_pickle_files/" + wellsPickleName)

			if mp == 'P91_92':
				function_get_pi_data_create_pickle_P9192()
			else:
				function_get_pi_data_create_pickle_pads(str(mp))

			df_all_data_pads = pd.read_pickle(rootDir + "/prod_pickle_files/" + padsPickleName)

		else:

			try:
				df_all_data_wells = pd.read_pickle(rootDir + "/prod_pickle_files/" + wellsPickleName)
			except:
				
				function_get_pi_data_create_pickle_wells(str(mp))
				df_all_data_wells = pd.read_pickle(rootDir + "/prod_pickle_files/" + wellsPickleName)

			try:
				df_all_data_pads = pd.read_pickle(rootDir + "/prod_pickle_files/" + padsPickleName)
			except:

				if mp == 'P91_92':
					function_get_pi_data_create_pickle_P9192()
				else:
					function_get_pi_data_create_pickle_pads(str(mp))

				df_all_data_pads = pd.read_pickle(rootDir + "/prod_pickle_files/" + padsPickleName)


		uniqueAttributes = df_all_data_wells['attribute'].unique()

		df_all_data_wells_esp_frequency = df_all_data_wells.loc[df_all_data_wells['attribute'] == 'esp_frequency']
		df_all_data_wells_esp_frequency_pivot = df_all_data_wells_esp_frequency.reset_index().pivot_table(index='date', columns='well', values='value')

		df_all_data_wells_temp_tubing = df_all_data_wells.loc[df_all_data_wells['attribute'] == 'temp_tubing']
		df_all_data_wells_temp_tubing_pivot = df_all_data_wells_temp_tubing.reset_index().pivot_table(index='date', columns='well', values='value')


		df_all_data_wells_casing_valve_test_status=df_all_data_wells.loc[df_all_data_wells['attribute'] == 'casing_valve']
		
		wells = df_all_data_wells_esp_frequency_pivot.columns

		df_all_data_pads_pg = df_all_data_pads.loc[df_all_data_pads['attribute'] == 'produced_gas']
		#df_all_data_pads_emulsion= df_all_data_pads.loc[df_all_data_pads['attribute'] == 'real_time_pad_flow']


		#script that will take the linear regression (rate of change) over different time periods for produced gas and store in a data frame
		# For each pad: filters produced_gas from pad-level data, looks back 2, 7, and 14 days, applies linear regression (linregress) to determine daily rate of change (slope), saves to a table
			
		#temp_df = pd.DataFrame(data={'mp': [mp]})
		df_roc_data_temp = pd.DataFrame()
		df_roc_data_temp['mp'] = [mp]
		for dayslookback in pg_roc_lookback_list:

			df_temp =df_all_data_pads_pg[~(df_all_data_pads_pg['date'] < (endDate - dateutil.relativedelta.relativedelta(days = dayslookback)))]


			iCount = 0
			step = 0.6
			timeHourlyStep = []
			for i in df_temp['date'].tolist():
				timeHourlyStep.append(step*iCount)
				iCount = iCount +1
			

			try:
				roc = linregress(timeHourlyStep, df_temp['value'].tolist()).slope
				df_roc_data_temp[str(dayslookback)+'d'] = [round(roc*24, 1)]
			except:
				df_roc_data_temp[str(dayslookback)+'d'] = 'err'


		df_pg_roc_data = pd.concat([df_pg_roc_data, df_roc_data_temp])

		


		trip_well_names = []

		#fig = go.Figure()
		fig = make_subplots(specs=[[{"secondary_y": True}]])

		fig.add_trace(go.Scatter(
			x=df_all_data_pads_pg['date'].tolist(),
			y=df_all_data_pads_pg['value'].tolist(),
			mode='lines',
			line=dict(
				color='black',
				),
			name='PG'),
			secondary_y=False, 
		)

		# fig.add_trace(go.Scatter(
		# 	x=df_all_data_pads_emulsion['date'].tolist(),
		# 	y=df_all_data_pads_emulsion['value'].tolist(),
		# 	mode='lines',
		# 	line=dict(
		# 		color='yellow',
		# 		),
		# 	name='Emulsion'),
		# 	secondary_y=True, 
		# )

		

		trip_dates = []
		trip_desc = []
		trip_pgVal = []

		start_dates=[]
		start_desc = []
		start_pgVal = []

		#df_casing_valve_postiion = pd.DataFrame(columns = ['well', '48hr']))
		# = pd.DataFrame(data={'well': trip_dates, 'vals': trip_desc})
		well_csg_valve_lookback_list = [2, 7, 14]
		df_csg_valve_data= pd.DataFrame()
		df_csg_valve_data['Well'] = ''
		for i in well_csg_valve_lookback_list:
			df_csg_valve_data[str(i) + 'd'] = nan
		
		tempThreasholdForFlowing = 140
		for well in wells:

			print(well)

			#get the 48hour average casing valve position and append to a dataframe
			df_csg = df_all_data_wells_casing_valve_test_status.loc[df_all_data_wells['well'] == well]

			df_csg_valve_data_temp = pd.DataFrame()
			df_csg_valve_data_temp['Well'] = [well]
			for dayslookback in well_csg_valve_lookback_list:

				df_temp =df_csg[~(df_csg['date'] < (endDate - dateutil.relativedelta.relativedelta(days = dayslookback)))]
				list_vals = df_temp['value'].tolist()
				try:
					avg_valve_pos = sum(list_vals)/len(list_vals)
				except:
					avg_valve_pos = 0
				df_csg_valve_data_temp[str(dayslookback)+'d'] = [round(avg_valve_pos, 1)]

			df_csg_valve_data = pd.concat([df_csg_valve_data, df_csg_valve_data_temp])


			# ESP event detection: loop through each well and each time step
			# Tracks ESP frequency changes to identify: Trips: sudden drops (delta <-20Hz), Starts: sudden jumps (delta > +20hz)
			# Deadhead/NFE: Checks if tubing temperature is dropping quickly while ESP is running , triggered by a 4-hr slope <-5degC and previous temp >140 degC 
			# Speedups: If ESP frequency gradually increases over 10 points (but less than +25hz total) is marked as a ramp-up 
			# stored in all_events and plotted in PG plot
			
			lastEventTrip = False #for deadehad logic
			deadheadEventDetected = False
			speedUpEventsDetected = False
			
			#x_anno = []
			#y_anno = []
			#text_anno = []

			for rowNum, (index, row) in enumerate(df_all_data_wells_esp_frequency_pivot.iterrows()):


				if rowNum ==0:
					prevVal = row[well]
				else:
					currentVal = row[well]
					diff = currentVal-prevVal

					if rowNum > 5 and deadheadEventDetected == True:
						#check to see if well has started flowing again
						#(established by the last 5 points being greater than flowing temp threashhold and then cancel deadhead event true so that it can be detected again)
						try:
							dateDiffLastDeadhead = df_all_data_wells_esp_frequency_pivot.index.tolist()[rowNum] - lastDeadheadDate
							daysDiff = dateDiffLastDeadhead.days
							tempList=[]
							#print(rowNum)
							#print(df_all_data_wells_temp_tubing_pivot[well])
							#print(len(df_all_data_wells_temp_tubing_pivot[well].tolist()))
							#print('list length above')

							#print('length of esp pivot table below')
							#print(df_all_data_wells_esp_frequency_pivot[well])
							#print('------')
							#print(df_all_data_wells_esp_frequency_pivot)

							for i in range(1,6):
								#print(rowNum)
								tempList.append(df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum-i])

							if all(i >= (tubingTempPriorToDeadhead-10) for i in tempList) and daysDiff >1:
								deadheadEventDetected = False
						except:
							deadheadEventDetected = False
						
						#print(tubingTempPriorToDeadhead)
						#print(tempList)
						#print('curr date', df_all_data_wells_esp_frequency_pivot.index.tolist()[rowNum])
						#print('lastdeadhead date', dateDiffLastDeadhead)
						#print(daysDiff)
						#print(deadheadEventDetected)


						
						#input('')
						


					if diff < -20:
						lastEventTrip = True
						deadheadEventDetected = False
						trip_desc_str = 'ESP Trip', index, well, ' ESP went from ', prevVal, 'hz to ', currentVal, 'hz'
						print(trip_desc_str)
						date = df_all_data_wells_esp_frequency_pivot.index.tolist()[rowNum]
						annotation = well + ' trip'
						#print(df_all_data_pads_pg)
						trip_dates.append(date)
						trip_desc.append(annotation)
						all_events.append({'well':well, 'date': date.strftime('%Y-%m-%d %X'), 'desc': annotation}) #############

						#x_anno.append(date)
						#y_anno.append(df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0])
						#text_anno.append(well)

						
						fig.add_trace(go.Scatter(
							x=[date], y=[df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0]],
							mode='markers + text',
							marker=dict(
								color='red',
								size=10,
							),
							text = [well],
							textposition="middle left",
							textfont=dict(
								family="sans serif",
								size=8,
								color='red'
							),
							marker_symbol = 'triangle-down', 
							name=annotation, 
							legendgroup = well),
							secondary_y=False,
						)

						# fig.add_annotation(text=well, x=date, y=df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0], showarrow=False, textangle=-90,
						# 	font=dict(
						# 		family="sans serif",
						# 		size=8,
						# 		color='red'
						# 	), yshift = -20)


					elif diff >20:
						lastEventTrip = False
						deadheadEventDetected = False
						trip_desc_str = 'ESP Start', index, well, ' ESP went from ', prevVal, 'hz to ', currentVal, 'hz'
						print(trip_desc_str)
						date = df_all_data_wells_esp_frequency_pivot.index.tolist()[rowNum]
						annotation = well + ' start'
						start_dates.append(date)
						start_desc.append(annotation)
						all_events.append({'well':well, 'date': date.strftime('%Y-%m-%d %X'), 'desc': annotation}) #############

						


						fig.add_trace(go.Scatter(
							x=[date], y=[df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0]],
							mode='markers + text',
							marker=dict(
								color='green',
								size=10,
							),
							text = [well],
							textposition="middle left",
							textfont=dict(
								family="sans serif",
								size=8,
								color='green'
							),
							marker_symbol = 'triangle-up',
							name=annotation, 
							legendgroup = well),
							secondary_y=False,
						)

						# fig.add_annotation(text=well, x=date, y=df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0], showarrow=False, textangle=-90,
						# 	font=dict(
						# 		family="sans serif",
						# 		size=8,
						# 		color='green'
						# 	), yshift = 20)

					
					elif lastEventTrip ==False: #no esp trip or start detected. Determine if deadhead event is occuring. 
						#create better logic. Right now the +40 = 4hrs at 6min data intervale
						if rowNum >40:
							if deadheadEventDetected != True:
								try: #it will not be able to perform this check on the first 4 hours of data.
									if df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum-40] > tempThreasholdForFlowing: #logic added so that wells that are offline with temps are not 
											
										temp_tubing_4hr_slope=(df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum] - df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum-40])/4
										if temp_tubing_4hr_slope <=-5:
											deadheadEventDetected = True
											tubingTempPriorToDeadhead = df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum-40]
											date = df_all_data_wells_temp_tubing_pivot.index.tolist()[rowNum-40]
											lastDeadheadDate = date
											#print(lastDeadheadDate)
											#input('')
											annotation = well + ' NFE'

											trip_dates.append(date)
											trip_desc.append(annotation)
											all_events.append({'well':well, 'date': date.strftime('%Y-%m-%d %X'), 'desc': annotation}) #############

											print('deadhead event found. ', 'current temp: ', df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum], '. temp @-4hrs: ', df_all_data_wells_temp_tubing_pivot[well].tolist()[rowNum-40], '. slope:', temp_tubing_4hr_slope, '. Date: ', date)

											fig.add_trace(go.Scatter(
												x=[date], y=[df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0]],
												mode='markers + text',
												marker=dict(
													color='blue',
													size=10,
												),
												text = [well],
												textposition="middle left",
												textfont=dict(
													family="sans serif",
													size=8,
													color='blue'
												),
												marker_symbol = 'x',
												name=annotation, 
												legendgroup = well),
												secondary_y=False,
											)							

											# fig.add_annotation(text=well, x=date, y=df_all_data_pads_pg['value'].loc[df_all_data_pads_pg['date']==date].tolist()[0], showarrow=False, textangle=-90,
											# 	font=dict(
											# 		family="sans serif",
											# 		size=8,
											# 		color='blue'
											# 	), yshift = 20)


								except:
									continue
					
					try:
						if rowNum >=10:
							if df_all_data_wells_esp_frequency_pivot[well].tolist()[rowNum] - df_all_data_wells_esp_frequency_pivot[well].tolist()[rowNum-10] >2:
								if speedUpEventsDetected ==False:
									#create new lists to enter data during the speed up event. 
									speedUpDates = []
									speedUpEmulsionVals = []
									speedUpESPSpeeds = []
									#this is the first detection and we will put in all 10pts used to make this realization that a speed up is occuring. Afterwards only the new point
									i = rowNum-10 
									while i <= rowNum:
										date = df_all_data_wells_esp_frequency_pivot.index.tolist()[i]
										speedUpDates.append(date)
										#speedUpEmulsionVals.append(df_all_data_pads_emulsion['value'].loc[df_all_data_pads_emulsion['date']==date].tolist()[0])
										speedUpESPSpeeds.append(df_all_data_wells_esp_frequency_pivot[well].tolist()[i])
										i=i+1
									speedUpEventsDetected = True
								elif speedUpEventsDetected ==True:
								
									date = df_all_data_wells_esp_frequency_pivot.index.tolist()[rowNum]
									speedUpDates.append(date)
									#speedUpEmulsionVals.append(df_all_data_pads_emulsion['value'].loc[df_all_data_pads_emulsion['date']==date].tolist()[0])
									speedUpESPSpeeds.append(df_all_data_wells_esp_frequency_pivot[well].tolist()[rowNum])

							else:
								if speedUpEventsDetected ==True:
									freqChange = speedUpESPSpeeds[len(speedUpESPSpeeds)-1]-speedUpESPSpeeds[0]
									if freqChange < 25: #this indicates a start up and it will already be captured in other anontations. 
										
										annotation = well + ' +' + str(round(freqChange,2)) + ' hz'
										print(annotation)
										#print('line prior to speed up print statement zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz')
										#print(annotation)
										#print(speedUpDates)
										#print(speedUpESPSpeeds)
										#print('line after to speed up print statement zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz')
										start_dates.append(speedUpDates[0])
										start_desc.append(annotation)
										all_events.append({'well':well, 'date': date.strftime('%Y-%m-%d %X'), 'desc': annotation}) #############

										
										#!!!!!!!!!!!!!!!!! - - uncomment to insert emulsion line to plot -- --!!!!!!!!!
										# fig.add_trace(go.Scatter(
										# 	x=[speedUpDates[0], speedUpDates[len(speedUpESPSpeeds)-1]],
										# 	y=[max(speedUpEmulsionVals), max(speedUpEmulsionVals)],
										# 	mode='lines',
										# 	line=dict(
										# 		color='red',
										# 		),
										# 	name=annotation),
										# 	secondary_y=True,
										# )
										#!!!!!!!!!!!!!!!!! - - uncomment to insert emulsion line to plot -- --!!!!!!!!!
									speedUpEventsDetected =False

					except:
						prevVal = currentVal
						continue

					prevVal = currentVal
		

		####Write main PG plot to png

		scale_input = 5

		fig.update_layout(width=600, height = 300, showlegend=False, margin=dict(l= 0, r= 0, t=0, b=0))
		fig.update_xaxes(range=[(endDate - dateutil.relativedelta.relativedelta(days = 14)), endDate])
		#fig.show()
		#fig.write_image(rootDir + "/prod_report_images/" + mp + "_pg_plot.png", format="png", scale=scale_input, engine="kaleido") 
		#fig.write_html(rootDir + "/prod_report_images/" + mp + "_pg_plot.html")
		
		#write tables
		#table generic set up inputs:
		header_height = 19
		header_font_size = 13
		cells_height=19
		cells_font_size=11

		#create trips / nfe table: 
		temp_sorting_df = pd.DataFrame(data={'date': trip_dates, 'vals': trip_desc})
		temp_sorting_df['date'] = pd.to_datetime(temp_sorting_df['date'])
		temp_sorting_df = temp_sorting_df[~(temp_sorting_df['date'] < (endDate - dateutil.relativedelta.relativedelta(days = 1)))]
		temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = True)
		outerWhileLoopCount = 0
		while len(temp_sorting_df) > 8: #need to search for group trips and / or delete
			outerWhileLoopCount = outerWhileLoopCount +1
			linesThatAreNotSingleTrips = 0 
			for index, row in temp_sorting_df.iterrows():
				if 'trip' in row['vals'] and 'trips' not in row['vals']: #trip detected
					indexListToDelete = []
					referenceDate = row['date']
					referenceIndex = index
					countOfNearbyTrips = 0	
					for index, row in temp_sorting_df.iterrows():
						if index != referenceIndex and 'trip' in row['vals'] and 'trips' not in row['vals']:
							dateDiff = row['date'] - referenceDate
							days, seconds = dateDiff.days, dateDiff.seconds
							hoursDiff = days * 24 + seconds // 3600
							if hoursDiff <=3:
								countOfNearbyTrips = countOfNearbyTrips +1
								indexListToDelete.append(index)
					if countOfNearbyTrips >0:
						indexListToDelete.append(referenceIndex)
						temp_sorting_df = temp_sorting_df.drop(indexListToDelete)
						temp = pd.DataFrame(data={'date': [referenceDate], 'vals': ["Trips (" + str(countOfNearbyTrips) + ")"]})
						#print(temp_sorting_df)
						#print(temp)
						#input('')
						temp_sorting_df=pd.concat([temp_sorting_df, temp])
						temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = True)
						break
				else:
					linesThatAreNotSingleTrips = linesThatAreNotSingleTrips +1
					if linesThatAreNotSingleTrips == len(temp_sorting_df) or outerWhileLoopCount == 8:
						temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = False)
						#print(temp_sorting_df)
						n=len(temp_sorting_df)-8+1
						temp_sorting_df = temp_sorting_df.iloc[:-n , :]
						temp = pd.DataFrame(data={'date': [endDate - dateutil.relativedelta.relativedelta(days = 1)], 'vals': ["Overflow..."]})
						temp_sorting_df=pd.concat([temp_sorting_df, temp])
						#print(temp_sorting_df)
						#input('lines were dropped from table')
						break




		temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = False)

		datestrs = [dt.datetime.strftime(x,'%b-%d %H:%M') for x in temp_sorting_df['date'].tolist()]

		values = []
		values.append(datestrs)
		values.append(temp_sorting_df['vals'].tolist())

		table_fig = go.Figure(data=[go.Table(
		columnorder = [1,2],
		columnwidth = [1,1.6],
		header = dict(
			values = ['Date', 'Event'],
			line_color='darkslategray',
			fill_color='royalblue',
			align=['center','center'],
			font=dict(color='white', size=header_font_size),
			height=header_height
		),
		cells=dict(
			values=values,
			line_color='darkslategray',
			fill=dict(color=['paleturquoise', 'white']),
			align=['left', 'center'],
			font_size=cells_font_size,
			height=cells_height)
			)
		])

		heightCalc = (len(datestrs) * cells_height) +header_height +2
		table_fig.update_layout(width=300, height = heightCalc, margin=dict(l= 0, r= 0, t=0, b=0))
		#table_fig.write_image(rootDir + "/prod_report_images/" + mp + "_trip_nfe.png", format="png", scale=scale_input, engine="kaleido") 

		#create starts/speed ups table: 
		temp_sorting_df = pd.DataFrame(data={'date': start_dates, 'vals': start_desc})
		temp_sorting_df['date'] = pd.to_datetime(temp_sorting_df['date'])
		temp_sorting_df = temp_sorting_df[~(temp_sorting_df['date'] < (endDate - dateutil.relativedelta.relativedelta(days = 1)))]
		temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = True)

		outerWhileLoopCount = 0
		while len(temp_sorting_df) > 8: #need to search for group trips and / or delete
			outerWhileLoopCount = outerWhileLoopCount +1

			linesThatAreNotSingleStarts = 0 
			for index, row in temp_sorting_df.iterrows():
				if 'start' in row['vals'] and 'starts' not in row['vals']: #trip detected
					indexListToDelete = []
					referenceDate = row['date']
					referenceIndex = index
					countOfNearbyStarts = 0	
					for index, row in temp_sorting_df.iterrows():
						if index != referenceIndex and 'start' in row['vals'] and 'starts' not in row['vals']:
							dateDiff = row['date'] - referenceDate
							days, seconds = dateDiff.days, dateDiff.seconds
							hoursDiff = days * 24 + seconds // 3600
							if hoursDiff <=3:
								countOfNearbyStarts = countOfNearbyStarts +1
								indexListToDelete.append(index)
					if countOfNearbyStarts >0:
						indexListToDelete.append(referenceIndex)
						temp_sorting_df = temp_sorting_df.drop(indexListToDelete)
						temp = pd.DataFrame(data={'date': [referenceDate], 'vals': ["Starts (" + str(countOfNearbyStarts) + ")"]})
						#print(temp_sorting_df)
						#print(temp)
						#input('')
						temp_sorting_df=pd.concat([temp_sorting_df, temp])
						temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = True)
						break
				else:
					linesThatAreNotSingleStarts = linesThatAreNotSingleStarts +1
					if linesThatAreNotSingleStarts == len(temp_sorting_df) or outerWhileLoopCount == 8:
						temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = False)
						#print(temp_sorting_df)
						#input('')
						n=len(temp_sorting_df)-8+1
						temp_sorting_df = temp_sorting_df.iloc[:-n , :]
						#temp_sorting_df = temp_sorting_df.drop(df.tail(len(temp_sorting_df)-(8+1)).index, inplace = True)
						#print(temp_sorting_df)
						temp = pd.DataFrame(data={'date': [endDate - dateutil.relativedelta.relativedelta(days = 1)], 'vals': ["Overflow..."]})
						temp_sorting_df=pd.concat([temp_sorting_df, temp])
						#print(temp_sorting_df)
						#input('lines were dropped from table')
						break


		temp_sorting_df = temp_sorting_df.sort_values(by='date', ascending = False)
		datestrs = [dt.datetime.strftime(x,'%b-%d %H:%M') for x in temp_sorting_df['date'].tolist()]

		values = []
		values.append(datestrs)
		values.append(temp_sorting_df['vals'].tolist())

		table_fig = go.Figure(data=[go.Table(
		columnorder = [1,2],
		columnwidth = [1,1.5],
		header = dict(
			values = ['Date', 'Event'],
			line_color='darkslategray',
			fill_color='royalblue',
			align=['center','center'],
			font=dict(color='white', size=header_font_size),
			height=header_height
		),
		cells=dict(
			values=values,
			line_color='darkslategray',
			fill=dict(color=['paleturquoise', 'white']),
			align=['center', 'center'],
			font_size=cells_font_size,
			height=cells_height)
			)
		])

		heightCalc = (len(datestrs) * cells_height) +header_height +2
		table_fig.update_layout(width=300, height = heightCalc, margin=dict(l= 0, r= 0, t=0, b=0))
		#table_fig.write_image(rootDir + "/prod_report_images/" + mp+ "_start_speedup.png", format="png", scale=scale_input, engine="kaleido") 

		#create overall rate of change table

		values = []
		for col in df_pg_roc_data.columns:
			values.append(df_pg_roc_data[col].tolist())
		
		#values.append(datestrs)
		#values.append(temp_sorting_df['vals'].tolist())

		table_fig = go.Figure(data=[go.Table(
		columnorder = [1,2,3,4],
		columnwidth = [1, 0.75, 0.75, 0.75],
		header = dict(
			values = df_pg_roc_data.columns,
			line_color='darkslategray',
			fill_color='royalblue',
			align=['center','center'],
			font=dict(color='white', size=header_font_size),
			height=header_height
		),
		cells=dict(
			values=values,
			line_color='darkslategray',
			fill=dict(color=['paleturquoise', 'white']),
			align=['center', 'center'],
			font_size=cells_font_size,
			height=cells_height)
			)
		])

		heightCalc = (len(df_pg_roc_data['mp']) * cells_height) +header_height +2
		table_fig.update_layout(width=250, height = heightCalc, margin=dict(l= 0, r= 0, t=0, b=0))
		#table_fig.write_image(rootDir + "/prod_report_images/mp_pg_roc.png", format="png", scale=scale_input, engine="kaleido") 

		#create casing valve average position table

		df_csg_valve_data = df_csg_valve_data.sort_values(by=str(well_csg_valve_lookback_list[0])+"d", ascending = False)
		n=len(df_csg_valve_data)-8
		df_csg_valve_data = df_csg_valve_data.iloc[:-n , :]


		values = []
		for col in df_csg_valve_data.columns:
			values.append(df_csg_valve_data[col].tolist())
		
		#values.append(datestrs)
		#values.append(temp_sorting_df['vals'].tolist())

		table_fig = go.Figure(data=[go.Table(
		columnorder = [1,2,3,4],
		columnwidth = [1, 1, 1, 1],
		header = dict(
			values = df_csg_valve_data.columns,
			line_color='darkslategray',
			fill_color='royalblue',
			align=['center','center'],
			font=dict(color='white', size=header_font_size),
			height=header_height
		),
		cells=dict(
			values=values,
			line_color='darkslategray',
			fill=dict(color=['paleturquoise', 'white']),
			align=['center', 'center'],
			font_size=cells_font_size,
			height=cells_height)
			)
		])

		heightCalc = (len(df_csg_valve_data['Well']) * cells_height) +header_height +2
		table_fig.update_layout(width=300, height = heightCalc, margin=dict(l= 0, r= 0, t=0, b=0))
		#table_fig.write_image(rootDir + "/prod_report_images/" + mp + "_csg_valve.png", format="png", scale=scale_input, engine="kaleido")
		# Open a file in write mode.
	
	print(all_events) 
	with open('events.csv', 'w') as f:
		# Write all the dictionary keys in a file with commas separated.
		f.write(','.join(all_events[0].keys()))
		f.write('\n') # Add a new line
		for row in all_events:
			# Write the values in a row.
			f.write(','.join(str(x) for x in row.values()))
			f.write('\n') # Add a new line

	

	print('images are created. Starting creating report to PDF.')
	#py_create_pdf_report.create_report()

	f = open(rootDir + "/Log.txt","a+")
	f.write("\r\n")
	str_msg = str(dt.datetime.today()) + " - Code is complete"
	f.write(str_msg + "\r\n")
	f.write("-----------------------------------\r\n")
	f.write("\r\n")
	f.close()
	print('script complete')
except:
	print(traceback.format_exc())
	f = open(rootDir + "/Log.txt","a+")
	f.write("\r\n")
	f.write(str(dt.datetime.today()) + "\r\n")
	f.write(str(traceback.format_exc()) + "\r\n")
	f.write("\r\n")
	f.write("-----------------------------------\r\n")
	f.close()


in try
plant_1
93FI-81150/PV.CV
93FI-81150/PV.CV
plant_2
99FI-40559/ALM1/PV.CV
99FI-40559/ALM1/PV.CV
plant_3
93FI-22203/ALM1/PV.CV
93FI-22203/ALM1/PV.CV
plant_4
92FI-2020/PV.CV
92FI-2020/PV.CV
plant_5
91FI-47408/PV.CV
91FI-47408/PV.CV
plant_6
91FI-13001/PV.CV
91FI-13001/PV.CV
plant_7
91FI-27408/PV.CV
91FI-27408/PV.CV
sru
91FC-1019/PID1/PV.CV
91FC-1019/PID1/PV.CV
field_emul
FB_TOTAL_EMULSION_CORRECTED
FB_TOTAL_EMULSION_CORRECTED
                    date      plant_1       plant_2   plant_3     plant_4  \
0    2025-04-07 05:55:00  2592.122070  2.522298e-06  0.002004  176.858047   
1    2025-04-07 06:01:00  2533.791504  0.000000e+00  0.488901  169.854492   
2    2025-04-07 06:07:00  2614.538086  2.271782e+02  0.258899  179.637253   
3    2025-04-07 06:13:00  2604.281982  3.337727e-07  1.561299  167.978241   
4    2025-04-07 06:19:00  2604.501465  5.377554e+02  3.630567  169.746094   
...                  ...          ...           ...       ...         ...   
3356 2025-04-21 05:31:00  219

C:\Users\MoBeaulieu\AppData\Local\Temp\ipykernel_6812\4199663753.py:55: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[49.63712692260742, 49.63712692260742]
105NSI-80620/AI1/PV.CV
[0.0007500174106098711, 0.0007500174106098711]
106NSI-80761/AI1/PV.CV
[41.64048385620117, 41.64048385620117]
106NSI-81061/AI1/PV.CV
[-0.0938250869512558, -0.0938250869512558]
106NSI-81461/AI1/PV.CV
[-0.25726619362831116, -0.25726619362831116]
108NSI-80261/AI1/PV.CV
[-0.1688496321439743, -0.1688496321439743]
108NSI-81561/AI1/PV.CV
[-2.4130003452301025, -2.4130003452301025]
116NSI-80361/AI1/PV.CV
[-0.07034274935722351, -0.07034274935722351]
116NSI-80461/AI1/PV.CV
[4.258569240570068, 4.258569240570068]
116NSI-80861/AI1/PV.CV
[-0.09613118320703506, -0.09613118320703506]
   well low_freq high_freq  current_freq
0   5P1       45        49          47.3
0   5P6       41        47          -0.0
0   5P7       42        56          52.2
0   5P8       44        48          -0.2
0  5P11       46        49          49.6
0   5N6       52        52           0.0
0   6P7       38        45          41.6
0  6P10       41        45          -